# Create `SSC_grid_georef.nc` File

Please see 
https://nbviewer.org/github/SalishSeaCast/analysis-doug/blob/main/notebooks/continental-HRDPS/crop-grib-to-SSC-domain.ipynb.ipynb
for the story behind why this file is needed.

In [1]:
from pathlib import Path
import textwrap

import arrow
import matplotlib.pyplot as plt
import xarray
import yaml

In [2]:
config = yaml.safe_load(
    textwrap.dedent(
        """\
        weather:
            download:
                2.5 km:
                    GRIB dir: /results/forcing/atmospheric/continental2.5/GRIB/
                    MSC file template: "{date}T{forecast}Z_MSC_HRDPS_{variable}_RLatLon0.0225_PT{hour}H.grib2"
                    SSC georef: /results/forcing/atmospheric/continental2.5/GRIB/SSC_grid_georef.nc
                    variables:
                        # [MSC name, GRIB std name, NEMO name]
                        - [UGRD_AGL-10m, u10, u_wind]           # u component of wind velocity at 10m elevation
                        - [VGRD_AGL-10m, v10, v_wind]           # v component of wind velocity at 10m elevation
                        - [DSWRF_Sfc, ssrd, solar]              # accumulated downward shortwave (solar) radiation at ground level
                        - [DLWRF_Sfc, strd, therm_rad]          # accumulated downward longwave (thermal) radiation at ground level
                        - [LHTFL_Sfc, lhtfl, LHTFL_surface]     # upward surface latent heat flux (for VHFR FVCOM)
                        - [TMP_AGL-2m, t2m, tair]               # air temperature at 2m elevation
                        - [SPFH_AGL-2m, sh2, qair]              # specific humidity at 2m elevation
                        - [RH_AGL-2m, r2, RH_2maboveground]     # relative humidity at 2m elevation (for VHFR FVCOM)
                        - [APCP_Sfc, unknown, precip]           # accumulated precipitation at ground level
                        - [PRATE_Sfc, prate, PRATE_surface]     # precipitation rate at ground level (for VHFR FVCOM)
                        - [PRMSL_MSL, prmsl, atmpres]           # atmospheric pressure at mean sea level
                    lon indices: [300, 490]
                    lat indices: [230, 460]
        """
    )
)

In [3]:
grib_dir = Path(config["weather"]["download"]["2.5 km"]["GRIB dir"])
msc_file_tmpl = config["weather"]["download"]["2.5 km"]["MSC file template"]
ssc_georef = config["weather"]["download"]["2.5 km"]["SSC georef"]
var_names = config["weather"]["download"]["2.5 km"]["variables"]
y_min, y_max = config["weather"]["download"]["2.5 km"]["lat indices"]
x_min, x_max = config["weather"]["download"]["2.5 km"]["lon indices"]
# We need 1 point more than the final domain size to facilitate calculation of the
# grid rotation angle for the wind components
y_slice = slice(y_min, y_max + 1)
x_slice = slice(x_min, x_max + 1)

In [4]:
fcst_date = arrow.get("2023-03-29")
fcst_yyyymmdd = fcst_date.format("YYYYMMDD")
forecast = "06"

In [5]:
ny, nx = y_max - y_min + 1, x_max - x_min + 1
for fcst_step in range(1, 2):
    grib_hr_dir = grib_dir / Path(fcst_yyyymmdd, forecast, f"{fcst_step:03d}")
    msc_var, grib_var, _ = var_names[0]
    msc_grib_file = msc_file_tmpl.format(
        date=fcst_yyyymmdd,
        forecast=forecast,
        variable=msc_var,
        hour=f"{fcst_step:03d}",
    )
    with xarray.open_dataset(
        grib_hr_dir/msc_grib_file, engine="cfgrib", backend_kwargs={"indexpath": ""}
    ) as msc_ds:
        ssc_ds = msc_ds.sel(y=y_slice, x=x_slice)
    ssc_ds[grib_var].attrs.update(
        {
            "GRIB_numberOfPoints": nx*ny,
            "GRIB_Nx": nx,
            "GRIB_Ny": ny,
        }
    )

ssc_ds


<xarray.Dataset>
Dimensions:            (y: 231, x: 191)
Coordinates:
    time               datetime64[ns] ...
    step               timedelta64[ns] ...
    heightAboveGround  float64 ...
    latitude           (y, x) float64 ...
    longitude          (y, x) float64 ...
    valid_time         datetime64[ns] ...
Dimensions without coordinates: y, x
Data variables:
    u10                (y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             cwao
    GRIB_centreDescription:  Canadian Meteorological Service - Montreal
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             Canadian Meteorological Service - Montreal
    history:                 2023-04-01T12:32 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
lon_da = xarray.DataArray(
    name="longitude",
    data=ssc_ds.longitude.data,
    coords={
        "y": ssc_ds.y,
        "x": ssc_ds.x,
    },
    attrs=ssc_ds.longitude.attrs,
)

lon_da


<xarray.DataArray 'longitude' (y: 231, x: 191)>
array([[-126.290664, -126.258888, -126.227104, ..., -120.250208,
        -120.2178  , -120.185384],
       [-126.295984, -126.264192, -126.232392, ..., -120.252792,
        -120.220368, -120.187936],
       [-126.301304, -126.269496, -126.23768 , ..., -120.255384,
        -120.222936, -120.190496],
       ...,
       [-127.628536, -127.593256, -127.557968, ..., -120.902584,
        -120.8664  , -120.830224],
       [-127.635032, -127.599736, -127.564432, ..., -120.90576 ,
        -120.869568, -120.83336 ],
       [-127.641536, -127.606224, -127.570904, ..., -120.908944,
        -120.872728, -120.836512]])
Coordinates:
  * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 222 223 224 225 226 227 228 229 230
  * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 182 183 184 185 186 187 188 189 190
Attributes:
    units:          degrees_east
    standard_name:  longitude
    long_name:      longitude

In [7]:
lat_da = xarray.DataArray(
    name="latitude",
    data=ssc_ds.latitude.data,
    coords={
        "y": ssc_ds.y,
        "x": ssc_ds.x,
    },
    attrs=ssc_ds.latitude.attrs,
)

lat_da


<xarray.DataArray 'latitude' (y: 231, x: 191)>
array([[46.123396, 46.127044, 46.130684, ..., 46.633464, 46.635216,
        46.636964],
       [46.145592, 46.149244, 46.152884, ..., 46.655892, 46.657648,
        46.659392],
       [46.167788, 46.17144 , 46.175084, ..., 46.678324, 46.68008 ,
        46.681824],
       ...,
       [51.176924, 51.180988, 51.18504 , ..., 51.745772, 51.747732,
        51.749684],
       [51.199052, 51.20312 , 51.207176, ..., 51.768188, 51.770148,
        51.7721  ],
       [51.22118 , 51.225248, 51.229308, ..., 51.7906  , 51.792564,
        51.794516]])
Coordinates:
  * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 222 223 224 225 226 227 228 229 230
  * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 182 183 184 185 186 187 188 189 190
Attributes:
    units:          degrees_north
    standard_name:  latitude
    long_name:      latitude

In [9]:
georef_ds = xarray.Dataset(
    data_vars={
        "longitude": lon_da,
        "latitude": lat_da,
    },
    coords={
        "y": ssc_ds.y,
        "x": ssc_ds.x,
    },
    attrs=ssc_ds.attrs,
)

georef_ds.attrs.pop("history")
georef_ds

<xarray.Dataset>
Dimensions:    (y: 231, x: 191)
Coordinates:
  * y          (y) int64 0 1 2 3 4 5 6 7 8 ... 223 224 225 226 227 228 229 230
  * x          (x) int64 0 1 2 3 4 5 6 7 8 ... 183 184 185 186 187 188 189 190
Data variables:
    longitude  (y, x) float64 -126.3 -126.3 -126.2 ... -120.9 -120.9 -120.8
    latitude   (y, x) float64 46.12 46.13 46.13 46.13 ... 51.79 51.79 51.79
Attributes:
    GRIB_edition:            2
    GRIB_centre:             cwao
    GRIB_centreDescription:  Canadian Meteorological Service - Montreal
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             Canadian Meteorological Service - Montreal

In [11]:
georef_ds.to_netcdf(ssc_georef)

In [13]:
!ls -lh /results/forcing/atmospheric/continental2.5/GRIB/SSC_grid_georef.nc

-rw-r--r-- 1 1557 210 701K Apr  1 12:35 /results/forcing/atmospheric/continental2.5/GRIB/SSC_grid_georef.nc
